In [37]:
from core.utils import load_metrics, generate_population, tournament_selection, binary_tournament
from core.crossover import fixed_crossover , ordered_crossover, partialMap_crossover
from core.mutation import mutate, multiple_mutate, inversion, scramble
from core.replacement import replace_firstweak, replace_weakest
from core.visualization import draw_cost, draw_cost_seperate
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn

In [38]:
sample = [
    [0,2,1,2,1],
    [2,0,2,2,2],
    [1,2,0,2,1],
    [2,2,2,0,2],
    [1,2,1,2,0]
]


In [39]:
from core import fitness
from typing import List, Callable

class Chromosome:
    def __init__(self, genes:List[int]):
        self.genes = genes
        self.phenomes = self.__fitness__(genes=genes)

    def __fitness__(self, genes:List[int]) -> int:
        return fitness(genes, distance_metric=sample)

    def __str__(self):
        return f"Genes: {self.genes}\nPhenomes: {self.phenomes:.5f}"

In [40]:
class Genetic(nn.Module):
    def __init__(self, distance_metric, p_mutate, p_crossover) -> None:
        super().__init__()
        self.distance_metric = distance_metric
        self.p_mutate = p_mutate
        self.p_crossover = p_crossover
    
    def search(self,cfg):
        # TODO : make search return everything we need to track upon experiencing the lab
        pass

    def forward(self,cfg):
        return self.search(cfg)
    
        

### EXP

In [41]:
def tour_size_manager(pop_size):
    if (pop_size >= 100):
        return [pop_size*0.1,pop_size*0.2,pop_size*0.3,pop_size*0.8]
    else:
        return [2,4,6,8]
max_generations = [100,1000,10000]
population_sizes = [50,100,200]
tour_size = [5,10,20]
crossover_functions = [fixed_crossover, ordered_crossover, partialMap_crossover]
mutate_functions = [mutate, multiple_mutate, inversion, scramble]
replace_functions = [replace_firstweak, replace_weakest]

In [42]:
# class Parameters:
#     def __init__(self,max_gen,pop_size,tour_size,cross_fn,mutate_fn,replace_fn) -> None:
#         self.max_gen = max_gen
#         self.pop_size = pop_size
#         self.tour_size = tour_size
#         self.cross_fn = cross_fn
#         self.mutate_fn = mutate_fn
#         self.replace_fn = replace_fn
from dataclasses import dataclass

@dataclass
class Parameters:
    max_gen : int
    pop_size : int
    tour_size : int
    cross_fn : Callable
    mutate_fn : Callable
    replace_fn :Callable

In [43]:
cfg = {
    "max_gen" : 100,
    "pop_size" : 50,
    "tour_size" : 5,
    "cross_fn" : fixed_crossover,
    "mutate_fn" : mutate,
    "replace_fn" : replace_firstweak
}

In [44]:
cfg1 = Parameters(100,50,5,fixed_crossover,mutate,replace_firstweak)

In [45]:
cfg1

Parameters(max_gen=100, pop_size=50, tour_size=5, cross_fn=<function fixed_crossover at 0x000002327E7CB7E0>, mutate_fn=<function mutate at 0x0000023206D45620>, replace_fn=<function replace_firstweak at 0x00000232072A49A0>)

In [52]:
import itertools
# Generate all combinations of parameters
parameter_combinations = list(itertools.product(
    max_generations, population_sizes, tour_size,
    crossover_functions, mutate_functions, replace_functions
))

# You can access each combination as a tuple in parameter_combinations
print("poissble way:",len(parameter_combinations))
params_list = []
for params in parameter_combinations:
    # print(Parameters(*params))
    params_list.append(Parameters(*params))
print(f"num res : {len(params_list)}")

poissble way: 648
num res : 648


In [56]:
parameter_combinations[0]

(100,
 50,
 5,
 <function core.crossover.fixed_crossover(arg1, arg2)>,
 <function core.mutation.mutate(candidate)>,
 <function core.replacement.replace_firstweak(population, candidate, distance_metric)>)

In [57]:
[x for x in range(2)]

[0, 1]

## TEST CLASS CORE

In [3]:
from version2.utils import create_parameter_list, generate_population, parents_selection

In [4]:
from version2.classes import Parameters, Chromosome

In [6]:
sample = [
    [0,2,1,2,1],
    [2,0,2,2,2],
    [1,2,0,2,1],
    [2,2,2,0,2],
    [1,2,1,2,0]
]

In [9]:
pop1 = generate_population(10,sample)
pop1

[Chromosome(gene=[4, 0, 1, 3, 2], phenome=0.125),
 Chromosome(gene=[3, 1, 4, 0, 2], phenome=0.125),
 Chromosome(gene=[1, 4, 0, 3, 2], phenome=0.1111111111111111),
 Chromosome(gene=[1, 2, 4, 0, 3], phenome=0.125),
 Chromosome(gene=[0, 4, 2, 1, 3], phenome=0.125),
 Chromosome(gene=[0, 3, 2, 1, 4], phenome=0.1111111111111111),
 Chromosome(gene=[3, 4, 2, 1, 0], phenome=0.1111111111111111),
 Chromosome(gene=[2, 4, 1, 0, 3], phenome=0.1111111111111111),
 Chromosome(gene=[1, 3, 2, 0, 4], phenome=0.125),
 Chromosome(gene=[3, 1, 2, 0, 4], phenome=0.125)]

In [15]:
parents = parents_selection(pop1,2)
parents

[Chromosome(gene=[3, 1, 2, 0, 4], phenome=0.125),
 Chromosome(gene=[2, 4, 1, 0, 3], phenome=0.1111111111111111)]

In [20]:
# crossover
def point_crossover(parent1, parent2):
    assert isinstance(parent1,Chromosome) and isinstance(parent2,Chromosome) , f"parent(s) should have instance of Chromosome class"
    n = len(parent1.gene)
    # point = np.random.randint(n)
    point = 2

    child1, child2 = [None]*n , [None]*n
    child1[point:] = parent2.gene[point:]
    child2[point:] = parent1.gene[point:]
    
    for i in range(point):
        if parent1.gene[i] not in child1:
            child1[i] = parent1[i]
        if parent2.gene[i] not in child2:
            child2[i] = parent2[i]
    
    remain1 = [x for x in parent1.gene if x not in child1]
    remain2 = [x for x in parent2.gene if x not in child2]

    for i in range(n):
        if child1[i] == None:
            child1[i] = remain1.pop(0)
        if child2[i] == None:
            child2[i] = remain2.pop(0)
    
    return child1 , child2

In [22]:
point_crossover(parents[0],parents[1])

([2, 4, 1, 0, 3], [1, 3, 2, 0, 4])

In [66]:
def ordered_crossover(parent1, parent2):
    assert isinstance(parent1,Chromosome) and isinstance(parent2,Chromosome) , f"parent(s) should have instance of Chromosome class"
    n = len(parent1.gene)
    child1 = [-1] * n
    child2 = [-1] * n

    start, end = np.sort(np.random.choice(n, 2, replace=False))
    
    child1[start:end+1] = parent2.gene[start:end+1]
    child2[start:end+1] = parent1.gene[start:end+1]

    mapping_range = list(set(range(n)) - set(range(start,end+1)))
    # print(mapping_range)
    for idx in mapping_range:
        if parent1.gene[idx] not in child1:
            child1[idx] = parent1.gene[idx]
        if parent2.gene[idx] not in child2:
            child2[idx] = parent2.gene[idx]

    remain1 = [x for x in parent1.gene if x not in child1]
    remain2 = [x for x in parent2.gene if x not in child2]

    child1 = [remain1.pop(0) if x == -1 else x for x in child1 ]
    child2 = [remain2.pop(0) if x == -1 else x for x in child2 ]

    return child1 , child2

In [67]:
ordered_crossover(parents[0],parents[1])

([3, 2, 1, 0, 4], [1, 4, 2, 0, 3])

In [51]:
parents

[Chromosome(gene=[3, 1, 2, 0, 4], phenome=0.125),
 Chromosome(gene=[2, 4, 1, 0, 3], phenome=0.1111111111111111)]

In [52]:
parents[0].gene[2:3+1]

[2, 0]

In [59]:
list(set(range(4+1)) - set(range(2,3+1)))

[0, 1, 4]

In [56]:
list(range(4+1))

[0, 1, 2, 3, 4]